In [ ]:
"""
Custom notebook to:
 - calculate and plot the IMFs of cluster simulations (combining simulations of the same type into
   the same IMF)
 - will calculate sinks that have finished accreting and plot that too.
 - plot the observed IMFs of Kroupa 2001 and Chabrier 2005
 - calculate the KS-statistic (Kolmogorov-Smirnov) between a purely compressive and purely solenoidal
   set of simulations.
 - plot the cumulative IMFs of each realisation including a combined total for each type (two plots,
   one for compressive, one for solenoidal.)

Requires 'clusterSink*.ev' to exist. One for each sink and each simulation.
(Note, it only really uses the very last one, since we are looking at the final masses of the sinks.)

Written by:
David Liptai, Monash University.
2015-2016
"""

In [ ]:
from matplotlib.pyplot import *
import numpy as np
from scipy.stats import ks_2samp as kstest
import itertools
import os
import subprocess
import time
%matplotlib
close('all')

utime       =  1.487E+13/60./60./24./365. 				# in years
t_ff        =  0.806588045/2.*utime						# in years
m_jup       =  0.0009546								# in solar masses

threshold = 1.0e-4
G 		  = 6.67e-8    #cgs
R_sun 	  = 6.963e10   #cm
year	  = 3.15569e7  #s
M_sun	  = 1.9891e33  #g
L_sun	  = 3.846e33   #erg/s
au		  = 1.49598e13 #cm
radius	  = 5*R_sun	   #accretion radius

#Plotting stuff nicely
from matplotlib import gridspec
from IPython.display import display, Math, Latex
import math
from math import sqrt, cos, sin, pi
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as ml
from matplotlib.ticker import FormatStrFormatter, LinearLocator, NullFormatter, NullLocator, MultipleLocator
import matplotlib.ticker
import matplotlib.colors
from matplotlib.font_manager import FontProperties
from matplotlib import rc, text
plt.close('all')
fig_width_pt = 504   #245.27        # Get this from LaTeX using \showthe\columnwidth
inches_per_pt = 1.0/72.27               # Convert pt to inches
golden_mean = (np.sqrt(5)-1.0)/2.0      # Aesthetic ratio
fig_width = fig_width_pt*inches_per_pt  # width in inches
fig_height =fig_width*golden_mean       # height in inches
fig_size = [fig_width,fig_height]

fontsize=18
#fig_size = [18,16]
params = {'backend': 'pdf',
          'axes.labelsize': fontsize,
          'lines.markersize': 4,
          'font.size': fontsize,
          'xtick.major.size':8,
          'xtick.minor.size':4,
          'ytick.major.size':8,
          'ytick.minor.size':4,
          'xtick.major.width':2,
          'ytick.major.width':2,
          'xtick.minor.width':1,
          'ytick.minor.width':1,
          'lines.markeredgewidth':1,
          'axes.linewidth':1.2,
          'legend.fontsize': fontsize-3,
          'xtick.labelsize': fontsize-2,
          'ytick.labelsize': fontsize-2,
          'savefig.dpi':200,
          'path.simplify':True,
          'font.family': 'serif',
          'font.serif':'Times',
          'text.latex.preamble': [r'\usepackage{amsmath}'],
          'text.usetex':True,
          'axes.color_cycle': ['b', 'lime', 'r', 'purple', 'g', 'c', 'm', 'orange', 'darkblue', \
                               'darkcyan', 'y','orangered','chartreuse','brown','deeppink','lightgreen', 'k'],
          #'font.serif':cm,
          'figure.figsize': fig_size}
plt.rcParams.update(params)
plt.clf()
gs = gridspec.GridSpec(1,1)
plt.close('all')

solenoidals = ['sol' +str(i) for i in range(1,8)]
mixed       = ['mix' +str(i) for i in range(1,8)]
compressives= ['comp'+str(i) for i in range(1,8)]

def BASH(command):
    return subprocess.check_output(command,shell=True).decode().strip()

dir_names = solenoidals+mixed+compressives
dir_home = BASH('echo $HOME')
dir_prefix = '/Volumes/dlip1/runs/'#dir_home+'/dlip1/sinkfiles/'

def sinks(dirname=BASH('pwd'),sim_titles=False):
    dirname=dirname+'/'
    simulation=''
    if sim_titles:
        simulation = dirname.split('/')[-2]
    try:
        simNO=BASH('ls '+dirname+'clusterSink*.ev | tail -1')
        simNO=simNO.strip('.ev')[-3:]
        print('Using clusterSink*'+simNO)
        nsinks_max=BASH('ls '+dirname+'clusterSink*'+simNO+'.ev | grep -c clusterSink')
    except subprocess.CalledProcessError:
        print('Warning: no sink files found.')
        simNO=''
        nsinks_max='0'
        return tuple([[None]*3]*4)
    print(nsinks_max+' files found.')
    nsinks_max = int(nsinks_max)+1

    ptmasses_all = []
    ptmasses_finished = []

    for i in range(1,nsinks_max):
        fname = dirname+'clusterSink{:0>4}'.format(i)+simNO+'.ev'
        try:
            data = np.loadtxt(fname,skiprows=1)
        except:
            try:
                data = np.genfromtxt(fname,skip_footer=1)
            except:
                print('failed to load file: '+fname)
        if len(np.shape(data)) > 1:
            time = data[:,0]*utime
            mass = data[:,4]
            ptmasses_all += [mass[-1]]

            THRESH=False
            if THRESH:
                mdot = mass[-1]-mass[-2]
                if mdot<=threshold:
                    ptmasses_finished += [mass[-1]]
            else:
                mdot = np.mean([(mass[j]-mass[j-1])/(time[j]-time[j-1]) for j in range(-1,-3,-2)])  # in solar masses per year
                M	 = mass[-1]
                luminosity = G*(M*M_sun)/(radius*au) * (mdot*(M_sun/year))
                #print(str(mdot).ljust(18),str(luminosity/L_sun).ljust(18))
                if luminosity<=1.e-20*L_sun:
                    ptmasses_finished += [mass[-1]]
#	ptmasses_finished =np.array(ptmasses_finished)
    Ndone=len(ptmasses_finished)
    if Ndone>0:
        print('all: ',len(ptmasses_all),'fin: ',len(ptmasses_finished))
        return ptmasses_all, ptmasses_finished
    else:
        print('all: ',len(ptmasses_all),'fin: ',0)
        return ptmasses_all, []

In [ ]:
starttime=time.clock()
solALL=[]
solFIN=[]
for i in solenoidals:
    print('='*50)
    print(i)
    ALL, FIN = sinks(dirname=dir_prefix+i)
    solALL+= ALL
    solFIN+= FIN

compALL=[]
compFIN=[]
for i in compressives:
    print('='*50)
    print(i)
    ALL, FIN = sinks(dirname=dir_prefix+i)
    compALL+= ALL
    compFIN+= FIN
print('time taken = ',time.clock()-starttime)

In [ ]:
# Make Kroupa 2001 and Chabrier 2005 IMFs

masses = 10**np.linspace(-2,1,501)

def chabIMF(m):
    imf = np.zeros(len(m))
    for i in range(len(m)):
        if m[i] > 1: imf[i] = 0.041*m[i]**(-1.35)
        if m[i] <=1: imf[i] = 0.093*np.exp(-(np.log10(m[i])-np.log10(0.2))**2/(2*0.55**2))
    return imf

def kroupaIMF(m):
    imf = np.zeros(len(m))
    for i in range(len(m)):
        if m[i] < 0.08:     imf[i] = 0.5125*m[i]**(-0.3)
        if 0.08<=m[i]<=0.5: imf[i] = 0.0410*m[i]**(-1.3)
        if m[i]>0.5:        imf[i] = 0.0205*m[i]**(-2.3)
    return imf*m*2

plt.close('all')
plt.figure()
plt.plot(masses,chabIMF(masses))
plt.plot(masses,kroupaIMF(masses))
plt.xscale('log')
plt.yscale('log')
plt.xlim(xmin=1e-3)
plt.show()

In [ ]:
prms = {'text.color':'white',
        'axes.facecolor':'black',
        'axes.edgecolor':'white',
        'axes.labelcolor':'white',
        'xtick.color':'white',
        'ytick.color':'white',
        'grid.color':'white',
        'savefig.transparent' : True
       }
plt.rcParams.update(prms)


# --Set up the bins for the IMFs ----------
# 	Left and right boundaries of the IMF
# 	(for both: all sinks, and, only sinks that have finished accreting) ???
# 	(on log axis)

#	bin_L  = np.log10(np.min(ptmasses_all))
#	bin_R  = np.log10(np.max(ptmasses_all))
bin_L  = -2.4
bin_R  =  0.2
# Choose the number of bin edges (ie no. bins + 1)
nbinsEdges = 15+1
nbins_all      = nbinsEdges
nbins_finished = nbinsEdges
bins_all      = 10**np.linspace(bin_L,bin_R,nbins_all)
binwidths_all = bins_all[1:]-bins_all[:-1]
bins_acc      = 10**np.linspace(bin_L,bin_R,nbins_finished)
binwidths_fin = bins_acc[1:]-bins_acc[:-1]
#fontsize_ticks=24
#fontsize_labels=24
#fontsize_legend=20
imfs={}

chab_scale = 200
kroupa_scale = chab_scale

chab_colour ='seagreen'
#kroupa_colour='darkslategray'
kroupa_colour='gold'


plt.close('all')
#
#==Sol================================================================================================================================
#
#--Initial mass function, not normalised-----------------------------------------
f1 = plt.figure(figsize=fig_size)
ax1 = f1.add_subplot(gs[0])
#ax1.tick_params(which='both',axis='both',labelsize=fontsize_ticks,color='k',length=6,width=2,pad=8)
plt.tick_params(which='both',axis='both',pad=8)
imfs['sol_all']=ax1.hist(solALL,      normed=False, bins=bins_all,facecolor='black',\
                         alpha=1.0, histtype='step', label='All sinks',hatch='////',edgecolor='royalblue')
if (len(solFIN)>0):
    imfs['sol_fin']=ax1.hist(solFIN, normed=False, bins=bins_acc,facecolor='blue', \
                             alpha=1.0, histtype='step', label='Sinks finished\n  accreting',\
                              hatch='\\\\\\',edgecolor='lightcoral')
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_ylim([3e-1,1e2])
ax1.set_xlim([2e-3,4e1])
ax1.set_ylabel('Number')
ax1.set_xlabel(r'Mass [$\mathrm{M}_{\odot}$]')
plt.plot(masses,chab_scale*chabIMF(masses),label = 'Chabrier 05',color=chab_colour,linestyle='-',linewidth=2)
plt.plot(masses,kroupa_scale*kroupaIMF(masses),label = 'Kroupa 01',color=kroupa_colour,linestyle='--',linewidth=2)
plt.legend(loc='upper right',frameon=False)
plt.title('Solenoidal')
plt.subplots_adjust(bottom = 0.16,wspace=0.0,hspace=0.0,right=0.95)
#plt.savefig('solimf.pdf')
plt.show()

#
#==Comp================================================================================================================================
#
#--Initial mass function, not normalised-----------------------------------------
f2= plt.figure(figsize=fig_size)
ax2 = f2.add_subplot(gs[0])
#ax2.tick_params(which='both',axis='both',labelsize=fontsize_ticks,color='k',length=6,width=2,pad=8)
plt.tick_params(which='both',axis='both',pad=8)
imfs['comp_all']=ax2.hist(compALL,     normed=False, bins=bins_all,facecolor='black',\
                          alpha=1.0, histtype='step', label='All sinks',hatch='////',edgecolor='royalblue')
if (len(compFIN)>0):
    imfs['comp_fin']=ax2.hist(compFIN, normed=False, bins=bins_acc,facecolor='blue', \
                              alpha=1.0, histtype='step', label='Sinks finished\n  accreting',\
                              hatch='\\\\\\',edgecolor='lightcoral')
xscale('log')
yscale('log')
ylim([3e-1,1e2])
xlim([2e-3,4e1])
ylabel('Number')
xlabel(r'Mass [$\mathrm{M}_{\odot}$]')
plt.plot(masses,chab_scale*chabIMF(masses),label = 'Chabrier 05',color=chab_colour,linestyle='-',linewidth=2)
plt.plot(masses,kroupa_scale*kroupaIMF(masses),label = 'Kroupa 01',color=kroupa_colour,linestyle='--',linewidth=2)
plt.legend(loc='upper right',frameon=False)
plt.title('Compressive')
#plt.tight_layout()
plt.subplots_adjust(bottom = 0.16,wspace=0.0,hspace=0.0,right=0.95)
plt.show()
#plt.savefig('compimf.pdf')

print('-'*50)
print('ks test of masses, not bin heights')
print('sol vs comp (all)',kstest(solALL,compALL))
print('sol vs comp (fin)',kstest(solFIN,compFIN))

In [ ]:
starttime=time.clock()
prms = {'text.color':'black',
        'axes.facecolor':'white',
        'axes.edgecolor':'black',
        'axes.labelcolor':'black',
        'xtick.color':'black',
        'ytick.color':'black',
        'grid.color':'black',
        'savefig.transparent' : False
       }
plt.rcParams.update(prms)

plt.close('all')

final_masses={'sol':np.zeros([len(solenoidals),len(bins_all)-1]),
              'comp':np.zeros([len(compressives),len(bins_all)-1])
             }

count = 0
for i in solenoidals:
    print('='*50)
    print(i)
    ALL, FIN = sinks(dirname=dir_prefix+i)
    a=plt.hist(ALL,normed=False,bins=bins_all,facecolor='black',histtype='step')
    final_masses['sol'][count,:] = a[0]
    count+=1

count = 0
for i in compressives:
    print('='*50)
    print(i)
    ALL, FIN = sinks(dirname=dir_prefix+i)
    a=plt.hist(ALL,normed=False,bins=bins_all,facecolor='black',histtype='step')
    final_masses['comp'][count,:] = a[0]
    count+=1

plt.close('all')   
print('time taken = ',time.clock()-starttime)

In [ ]:
#
# Plot the cumulative IMFs
#

prms = {'text.color':'black',
        'axes.facecolor':'white',
        'axes.edgecolor':'black',
        'axes.labelcolor':'black',
        'xtick.color':'black',
        'ytick.color':'black',
        'grid.color':'black',
        'savefig.transparent' : False
       }
plt.rcParams.update(prms)

CUimf_comp=np.cumsum(imfs['comp_all'][0])
CUimf_sol=np.cumsum(imfs['sol_all'][0])
x=bins_all[:-1]
plt.close('all')

plt.figure()
plt.step(x,CUimf_sol/CUimf_sol[-1],linestyle='-',lw=3,c='green',where='post',label='Total (solenoidal)')
for hist in final_masses['sol']:
    y = np.cumsum(hist)
    plt.step(x,y/y[-1],linestyle='-',lw=1,c='mediumseagreen',where='post')
plt.ylim([0,1])
plt.xlim([1e-3,10])
plt.xscale('log')
plt.ylabel('Cumulative fractional IMF')
plt.xlabel(r'Mass [$\mathrm{M}_{\odot}$]')
plt.legend(loc='upper left',frameon=False)
plt.tight_layout()
plt.show()
#plt.savefig(dir_home+'/Desktop/cIMF_sol.pdf')

plt.figure()
plt.step(x,CUimf_comp/CUimf_comp[-1],linestyle='-',lw=3,c='red',where='post',label='Total (compressive)')
for hist in final_masses['comp']:
    y = np.cumsum(hist)
    plt.step(x,y/y[-1],linestyle='-',lw=1,c='salmon',where='post')
plt.ylim([0,1])
plt.xlim([1e-3,10])
plt.xscale('log')
plt.ylabel('Cumulative fractional IMF')
plt.xlabel(r'Mass [$\mathrm{M}_{\odot}$]')
plt.legend(loc='upper left',frameon=False)
plt.tight_layout()
plt.show()
#plt.savefig(dir_home+'/Desktop/cIMF_comp.pdf')